<img src="../../images/BikeDNA_logo.svg" width="250"  alt="BikeDNA logo" style="display:block; margin-left: auto; margin-right: auto;">
<a href="https://github.com/anerv/BikeDNA">Github</a>

# 1b. Intrinsic Analysis of OSM Data
This notebook analyzes the quality of OSM bicycle infrastructure data for a given area. The quality assessment is *intrinsic*, i.e. based only on the one input data set without makeing use of external information. For an extrinsic quality assessment that compares the OSM data to a user-provided reference data set, see the notebooks 3a and 3b.

The analysis assesses the *fitness for purpose* ([Barron et al., 2014](https://onlinelibrary.wiley.com/doi/10.1111/tgis.12073)) of OSM data for a given area. Outcomes of the analysis can be relevant for bicycle planning and research - especially for projects that include a network analysis of bicycle infrastructure, in which case the topology of the geometries is of particular importance.

Since the assessment does not make use of an external reference data set as the ground truth, no universal claims of data quality can be made. The idea is rather to enable those working with OSM-based bicycle networks to assess whether the data are good enough for their particular use case. The analysis assists in finding potential data quality issues but leaves the final interpretation of the results to the user.

The notebook makes use of quality metrics from a range of previous projects investigating OSM/VGI data quality, such as [Ferster et al. (2020)](https://www.tandfonline.com/doi/full/10.1080/15568318.2018.1519746), [Hochmair et al. (2015)](https://onlinelibrary.wiley.com/doi/abs/10.1111/tgis.12081), [Barron et al. (2014)](https://onlinelibrary.wiley.com/doi/10.1111/tgis.12073), and [Neis et al. (2012](https://www.mdpi.com/1999-5903/4/1/1)).

<div class="alert alert-block alert-info">
<b>Prerequisites &amp; Input/Output</b>
<p>
Run notebook 1a in advance.  
    
Output files of this notebook (data, maps, plots) are saved to the <span style="font-family:courier;">../results/OSM/[study_area]/</span> subfolders.
</p>
</div>

<div class="alert alert-block alert-info">
<b>Familiarity required</b>
<p>
For a correct interpretation of some of the metrics for spatial data quality, some familiarity with the area is necessary.
</p>
</div>

**Sections**
* [Data completeness](#Data-completeness)
    * [Network density](#Network-density)
* [OSM tag analysis](#OSM-tag-analysis)
    * [Missing tags](#Missing-tags)
    * [Incompatible tags](#Incompatible-tags)
    * [Tagging patterns](#Tagging-patterns)
* [Network topology](#Network-topology)
    * [Simplification outcome](#Simplification-outcome)
    * [Dangling nodes](#Dangling-nodes)
    * [Under/overshoots](#Under/overshoots)
    * [Missing intersection nodes](#Missing-intersection-nodes)
* [Network components](#Network-components)
    * [Disconnected components](#Disconnected-components)
    * [Components per grid cell](#Components-per-grid-cell)
    * [Component length distribution](#Component-length-distribution)
    * [Largest connected component](#Largest-connected-component)
    * [Missing links](#Missing-links)
    * [Component connectivity](#Component-connectivity)
* [Summary](#Summary)

<br />

In [ ]:
# Load libraries, settings and data

import json
import pickle
import warnings
from collections import Counter
import math

import contextily as cx
import folium
import geopandas as gpd
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
import yaml
from matplotlib import cm, colors

from src import evaluation_functions as eval_func
from src import plotting_functions as plot_func

%run ../settings/yaml_variables.py
%run ../settings/plotting.py
%run ../settings/tiledict.py
%run ../settings/load_osmdata.py
%run ../settings/df_styler.py
%run ../settings/paths.py

warnings.filterwarnings("ignore")

grid = osm_grid

## Data completeness

### Network density

In this setting, network density refers to the length of edges or number of nodes per km2. This is the usual definition of network density in spatial (road) networks, which is distinct from the *structural* network density known more generally in network science. Without comparing to a reference data set, network density does not in itself indicate spatial data quality. For anyone familiar with the study area, network density can however indicate whether parts of the area appear to be under- or over-mapped.

**Method**

The density here is not based on the geometric length of edges, but instead on the computed length of the infrastructure. For example, a 100-meter-long bidirectional path contributes with 200 meters of bicycle infrastructure. This method is used to take into account different ways of mapping bicycle infrastructure, which otherwise can introduce large deviations in network density. With `compute_network_density`, the number of elements (nodes, dangling nodes, and total infrastructure length) per unit area is calculated. The density is computed twice: first for the study area for both the entire network ('global density'), then for each of the grid cells ('local density'). Both global and local densities are computed for the entire network and for protected and unprotected infrastructure.

**Interpretation**

Since the analysis conducted here is intrinsic, i.e. it makes no use of external information, it cannot be known whether a low-density value is due to incomplete mapping, or due to actual lack of infrastructure in the area. However, a comparison of the grid cell density values can provide some insights, for example:
* lower-than-average infrastructure density indicates a locally sparser network
* higher-than-average node density indicates that there are relatively many intersections in a grid cell
* higher-than-average dangling node density indicates that there are relatively many dead ends in a grid cell

#### Global network density

<br />

In [ ]:
# Entire study area
edge_density, node_density, dangling_node_density = eval_func.compute_network_density(
    (osm_edges_simplified, osm_nodes_simplified),
    grid.unary_union.area,
    return_dangling_nodes=True,
)

density_results = {}
density_results["edge_density_m_sqkm"] = edge_density
density_results["node_density_count_sqkm"] = node_density
density_results["dangling_node_density_count_sqkm"] = dangling_node_density

osm_protected = osm_edges_simplified.loc[osm_edges_simplified.protected == "protected"]
osm_unprotected = osm_edges_simplified.loc[
    osm_edges_simplified.protected == "unprotected"
]
osm_mixed = osm_edges_simplified.loc[osm_edges_simplified.protected == "mixed"]

osm_data = [osm_protected, osm_unprotected, osm_mixed]
labels = ["protected_density", "unprotected_density", "mixed_density"]

for data, label in zip(osm_data, labels):
    if len(data) > 0:
        osm_edge_density_type, _ = eval_func.compute_network_density(
            (data, osm_nodes_simplified),
            grid.unary_union.area,
            return_dangling_nodes=False,
        )
        density_results[label + "_m_sqkm"] = osm_edge_density_type
    else:
        density_results[label + "_m_sqkm"] = 0

protected_edge_density = density_results["protected_density_m_sqkm"]
unprotected_edge_density = density_results["unprotected_density_m_sqkm"]
mixed_protection_edge_density = density_results["mixed_density_m_sqkm"]

print(f"For the entire study area, there are:")
print(f"- {edge_density:.2f} meters of bicycle infrastructure per km2.")
print(f"- {node_density:.2f} nodes in the bicycle network per km2.")
print(
    f"- {dangling_node_density:.2f} dangling nodes in the bicycle network per km2."
)
print(
    f"- {protected_edge_density:.2f} meters of protected bicycle infrastructure per km2."
)
print(
    f"- {unprotected_edge_density:.2f} meters of unprotected bicycle infrastructure per km2."
)
print(
    f"- {mixed_protection_edge_density:.2f} meters of mixed protection bicycle infrastructure per km2."
)

In [ ]:
# Save stats to csv
pd.DataFrame(
    {
        "metric": [
            "meters of bicycle infrastructure per square km",
            "nodes in the bicycle network per square km",
            "dangling nodes in the bicycle network per square km",
            "meters of protected bicycle infrastructure per square km",
            "meters of unprotected bicycle infrastructure per square km",
            "meters of mixed protection bicycle infrastructure per square km",
        ],
        "value": [
            np.round(edge_density, 2),
            np.round(node_density, 2),
            np.round(dangling_node_density, 2),
            np.round(protected_edge_density, 2),
            np.round(unprotected_edge_density, 2),
            np.round(mixed_protection_edge_density, 2),
        ],
    }
).to_csv(osm_results_data_fp + "stats_area.csv", index=False)

#### Local network density

In [ ]:
# Per grid cell
results_dict = {}
data = (osm_edges_simp_joined, osm_nodes_simp_joined.set_index("osmid"))

[
    eval_func.run_grid_analysis(
        grid_id,
        data,
        results_dict,
        eval_func.compute_network_density,
        grid["geometry"].loc[grid.grid_id == grid_id].area.values[0],
        return_dangling_nodes=True,
    )
    for grid_id in grid_ids
]

results_df = pd.DataFrame.from_dict(results_dict, orient="index")
results_df.reset_index(inplace=True)
results_df.rename(
    columns={
        "index": "grid_id",
        0: "osm_edge_density",
        1: "osm_node_density",
        2: "osm_dangling_node_density",
    },
    inplace=True,
)

grid = eval_func.merge_results(grid, results_df, "left")

osm_protected = osm_edges_simp_joined.loc[
    osm_edges_simp_joined.protected == "protected"
]
osm_unprotected = osm_edges_simp_joined.loc[
    osm_edges_simp_joined.protected == "unprotected"
]
osm_mixed = osm_edges_simp_joined.loc[osm_edges_simp_joined.protected == "mixed"]

osm_data = [osm_protected, osm_unprotected, osm_mixed]

osm_labels = ["osm_protected_density", "osm_unprotected_density", "osm_mixed_density"]

for data, label in zip(osm_data, osm_labels):
    if len(data) > 0:
        results_dict = {}
        data = (osm_edges_simp_joined.loc[data.index], osm_nodes_simp_joined)
        [
            eval_func.run_grid_analysis(
                grid_id,
                data,
                results_dict,
                eval_func.compute_network_density,
                grid["geometry"].loc[grid.grid_id == grid_id].area.values[0],
            )
            for grid_id in grid_ids
        ]

        results_df = pd.DataFrame.from_dict(results_dict, orient="index")
        results_df.reset_index(inplace=True)
        results_df.rename(columns={"index": "grid_id", 0: label}, inplace=True)
        results_df.drop(1, axis=1, inplace=True)

        grid = eval_func.merge_results(grid, results_df, "left")

In [ ]:
# Network density grid plots
         
set_renderer(renderer_map)
plot_cols = ["osm_edge_density", "osm_node_density", "osm_dangling_node_density"]
plot_titles = [
    area_name+": OSM edge density",
    area_name+": OSM node density",
    area_name+": OSM dangling node density",
]
filepaths = [
    osm_results_static_maps_fp + "density_edge_osm",
    osm_results_static_maps_fp + "density_node_osm",
    osm_results_static_maps_fp + "density_danglingnode_osm",
]
cmaps = [pdict["pos"], pdict["pos"], pdict["pos"]]
no_data_cols = ["count_osm_edges", "count_osm_nodes", "count_osm_nodes"]

plot_func.plot_grid_results(
    grid=grid,
    plot_cols=plot_cols,
    plot_titles=plot_titles,
    filepaths=filepaths,
    cmaps=cmaps,
    alpha=pdict["alpha_grid"],
    cx_tile=cx_tile_2,
    no_data_cols=no_data_cols,
)

#### Densities of protected and unprotected infrastructure

In BikeDNA, *protected infrastructure* refers to all bicycle infrastructure which is either separated from car traffic by for example an elevated curb, bollards, or other physical barriers, or for cycle tracks that are not adjacent to a street.

*Unprotected infrastructure* are all other types of lanes that are dedicated for bicyclists, but which only are separated by car traffic by e.g., a painted line on the street.

In [ ]:
# Network density grid plots

set_renderer(renderer_map)
plot_cols = ["osm_protected_density", "osm_unprotected_density", "osm_mixed_density"]
plot_titles = [
    area_name+": OSM protected infrastructure density (m/km2)",
    area_name+": OSM unprotected infrastructure density for (m/km2)",
    area_name+": OSM mixed protection infrastructure density (m/km2)",
]
filepaths = [
    osm_results_static_maps_fp + "density_protected_osm",
    osm_results_static_maps_fp + "density_unprotected_osm",
    osm_results_static_maps_fp + "density_mixed_osm",
]

cmaps = [pdict["pos"]] * len(plot_cols)
no_data_cols = ["osm_protected_density", "osm_unprotected_density", "osm_mixed_density"]
norm_min = [0] * len(plot_cols)
norm_max = [max(grid[plot_cols].fillna(value=0).max())] * len(plot_cols)

plot_func.plot_grid_results(
    grid=grid,
    plot_cols=plot_cols,
    plot_titles=plot_titles,
    filepaths=filepaths,
    cmaps=cmaps,
    alpha=pdict["alpha_grid"],
    cx_tile=cx_tile_2,
    no_data_cols=no_data_cols,
    use_norm=True,
    norm_min=norm_min,
    norm_max=norm_max,
)

## OSM tag analysis

For many practical and research purposes, more information than just the presence/absence of bicycle infrastructure is of interest. Information about e.g. the width of the infrastructure, speed limits, streetlights, etc. can be of high relevance, for example when evaluating the bike friendliness of an area or an individual network segment. The presence of these tags describing attributes of the bicycle infrastructure is however highly unevenly distributed in OSM, which poses a barrier to evaluations of bikeability and traffic stress. Likewise, the lack of restrictions on how OSM features can be tagged sometimes result in conflicting tags which can undermine the evaluation of cycling conditions.

This section includes analyzes of missing tags (edges with tags that lack information), incompatible tags (edges with tags labelled with two or more contradictory tags), and tagging patterns (the spatial variation of which tags are being used to describe bicycle infrastructure).

For the evaluation of tags, the non-simplified edges should be used to avoid issues with tags that have been aggregated in the simplification process.

### Missing tags

The information that is required or desirable to obtain from the OSM tags depends on the use case - for example, the tag `lit` for a project that studies light conditions on cycle paths. The workflow below allows to quickly analyze the percentage of network edges that have a value available for the tag of interest. 

**Method**

We analyze all tags of interest as defined in the `existing_tag_analysis` section of `config.yml`. For each of these tags, `analyze_existing_tags` is used to compute the total number and the percentage of edges that have a corresponding tag value.

**Interpretation**

On the study area level, a higher percentage of existing tag values indicates in principle a higher quality of the data set. However, this is different from an estimation of whether the existing tag values are truthful. On the grid cell level, lower-than-average percentages for existing tag values can indicate a more poorly mapped area. However, the percentages are less informative for grid cells with a low number of edges: for example, if a cell contains one single edge that has a tag value for `lit`, the percentage of existing tag values is 100% - but given that there is only 1 data point, this is less informative than, say, a value of 80% for a cell that contains 200 edges.

<br />

<div class="alert alert-block alert-info">
<b>User configurations</b>
<br>
<br>
In the analysis of OSM tags, user settings are used for:
<br>
<ul>
  <li>defining the tags to analyze for missing tags (<span style="font-family: monospace">missing_tag_analysis</span>) </li>
  <li>defining incompatible tag value combinations (<span style="font-family: monospace">incompatible_tags_analysis</span>) </li>
  <li>the visualization of tagging of bicycle infrastructure (<span style="font-family: monospace">bicycle_infrastructure_queries</span>) </li>
</ul>
</div>

#### Global missing tags

In [ ]:
print(f"Analysing tags describing:")
for k in existing_tag_dict.keys():
    print(k, "-", end=" ")
print("\n")

existing_tags_results = eval_func.analyze_existing_tags(osm_edges, existing_tag_dict)

for key, value in existing_tags_results.items():
    print(
        f"{key}: {value['count']} out of {len(osm_edges)} edges ({value['count']/len(osm_edges)*100:.2f}%) have information."
    )
    print(
        f"{key}: {round(value['length']/1000)} out of {round(osm_edges.geometry.length.sum()/1000)} km ({100*value['length']/osm_edges.geometry.length.sum():.2f}%) have information."
    )
    print("\n")

results_dict = {}
[
    eval_func.run_grid_analysis(
        grid_id,
        osm_edges_joined,
        results_dict,
        eval_func.analyze_existing_tags,
        existing_tag_dict,
    )
    for grid_id in grid_ids
]

# compute the length of osm edges in each grid cell to use for pct missing tags based on length
grid_feature_len = eval_func.length_features_in_grid(osm_edges_joined, "osm_edges")
grid = eval_func.merge_results(grid, grid_feature_len, "left")


restructured_results = {}

for key, value in results_dict.items():

    unpacked_results = {}

    for tag, _ in existing_tag_dict.items():

        unpacked_results[tag + "_count"] = value[tag]["count"]
        unpacked_results[tag + "_length"] = value[tag]["length"]

    restructured_results[key] = unpacked_results

results_df = pd.DataFrame.from_dict(restructured_results, orient="index")
cols = results_df.columns
new_cols = ["existing_tags_" + c for c in cols]
results_df.columns = new_cols
results_df["existing_tags_sum"] = results_df[new_cols].sum(axis=1)
results_df.reset_index(inplace=True)
results_df.rename(columns={"index": "grid_id"}, inplace=True)

grid = eval_func.merge_results(grid, results_df, "left")

for c in new_cols:
    if "count" in c:
        grid[c + "_pct"] = round(grid[c] / grid.count_osm_edges * 100, 2)
        grid[c + "_pct_missing"] = 100 - grid[c + "_pct"]

    elif "length" in c:
        grid[c + "_pct"] = round(grid[c] / grid.length_osm_edges * 100, 2)
        grid[c + "_pct_missing"] = 100 - grid[c + "_pct"]

existing_tags_pct = {}

existing_tags_pct = {}

for k, v in existing_tags_results.items():

    pct_dict = {}

    pct_dict["count_pct"] = (v["count"] / len(osm_edges)) * 100
    pct_dict["length_pct"] = (v["length"] / osm_edges.geometry.length.sum()) * 100

    existing_tags_pct[k] = pct_dict

for k, v in existing_tags_results.items():
    merged_results = v | existing_tags_pct[k]

    existing_tags_results[k] = merged_results

In [ ]:
# Save to csv

# Initialize lists with values for entire data set
edgecount_list = [len(osm_edges)]
kmcount_list = [round(osm_edges.geometry.length.sum() / 1000)]
keys_list = ["TOTAL"]

for key, value in existing_tags_results.items():
    keys_list.append(key)
    edgecount_list.append(value["count"])
    kmcount_list.append(round(value["length"] / 1000))

pd.DataFrame(
    {"tag": keys_list, "edge count": edgecount_list, "km (rounded)": kmcount_list}
).to_csv(osm_results_data_fp + "stats_tags.csv", index=False)

#### Local missing tags

In [ ]:
#  Plot pct missing for count

set_renderer(renderer_map)
count_cols = [c for c in cols if 'count' in c]
filepaths = [osm_results_static_maps_fp + "tagsmissing_" + c + '_osm' for c in count_cols]
plot_cols = ["existing_tags_" + c + "_pct_missing" for c in count_cols]
count_cols = [c.removesuffix('_count') for c in count_cols]
plot_titles = [area_name+": percent of missing OSM tags for: " + c for c in count_cols]

cmaps = [pdict["neg"]] * len(plot_cols)
no_data_cols = ["count_osm_edges"] * len(plot_cols)

plot_func.plot_grid_results(
    grid=grid,
    plot_cols=plot_cols,
    plot_titles=plot_titles,
    filepaths=filepaths,
    cmaps=cmaps,
    alpha=pdict["alpha_grid"],
    cx_tile=cx_tile_2,
    no_data_cols=no_data_cols,
)

In [ ]:
#  Plot pct missing based on the length of the edges with missing tags

set_renderer(renderer_map)
length_cols = [c for c in cols if 'length' in c]
plot_titles = [area_name+": percent of missing OSM tags for: " + c for c in length_cols]
filepaths = [osm_results_static_maps_fp + "tagsmissing_" + c + '_osm' for c in length_cols]
plot_cols = ["existing_tags_" + c + "_pct_missing" for c in length_cols]
plot_titles = [c.replace("_", " (") for c in plot_titles]
plot_titles = [c + ")" for c in plot_titles]

cmaps = [pdict["neg"]] * len(plot_cols)
no_data_cols = ["count_osm_edges"] * len(plot_cols)

plot_func.plot_grid_results(
    grid=grid,
    plot_cols=plot_cols,
    plot_titles=plot_titles,
    filepaths=filepaths,
    cmaps=cmaps,
    alpha=pdict["alpha_grid"],
    cx_tile=cx_tile_2,
    no_data_cols=no_data_cols,
)

### Incompatible tags

Given that the tags in OSM data lack coherency at times and there are no restrictions in the tagging process (cf. [Barron et al., 2014](https://onlinelibrary.wiley.com/doi/10.1111/tgis.12073)), incompatible tags might be present in the data set. For example, an edge might be tagged with the following two contradicting key-value pairs: `bicycle_infrastructure = yes` and `bicycle = no`. 

**Method**

In the `config.yml` file, a list of incompatible key-value pairs for tags in the `incompatible_tags_analysis` is defined. Since there is no limitation to which tags a data set could potentially contain, the list is, by definition, non-exhaustive, and can be adjusted by the user. In the section below, `check_incompatible_tags` is run, which identifies all incompatibility instances for a given area, first on the study area level and then on the grid cell level.

**Interpretation**

Incompatible tags are an undesired feature of the data set and render the corresponding data points invalid; there is no straightforward way to resolve the arising issues automatically, making it necessary to either correct the tag manually or to exclude the data point from the data set. A higher-than-average number of incompatible tags in a grid cell suggests local mapping issues.

#### Global incompatible tags (total number)

In [ ]:
incompatible_tags_results = eval_func.check_incompatible_tags(
    osm_edges, incompatible_tags_dict, store_edge_ids=True
)

print(
    f"In the entire data set, there are {sum(len(lst) for lst in incompatible_tags_results.values())} incompatible tag combinations (of those defined in the configuration file)."
)

results_dict = {}
[
    eval_func.run_grid_analysis(
        grid_id,
        osm_edges_joined,
        results_dict,
        eval_func.check_incompatible_tags,
        incompatible_tags_dict,
    )
    for grid_id in grid_ids
]

results_df = pd.DataFrame(
    index=results_dict.keys(), columns=results_dict[list(results_dict.keys())[0]].keys()
)

for i in results_df.index:
    for j in results_df.columns:
        results_df.at[i, j] = results_dict[i][j]

cols = results_df.columns
new_cols = ["incompatible_tags_" + c for c in cols]
results_df.columns = new_cols
results_df["incompatible_tags_sum"] = results_df[new_cols].sum(axis=1)
results_df.reset_index(inplace=True)
results_df.rename(columns={"index": "grid_id"}, inplace=True)
grid = eval_func.merge_results(grid, results_df, "left")

In [ ]:
# Save to csv
df = pd.DataFrame(
    index=incompatible_tags_results.keys(), data=incompatible_tags_results.values()
)
edge_ids = []
for index, row in df.iterrows():
    edge_ids.append(row.to_list())

df["edge_id"] = edge_ids
incomp_df = df[["edge_id"]]

incomp_df.to_csv(osm_results_data_fp + "incompatible_tags.csv", index=True)

#### Local incompatible tags (per grid cell)

In [ ]:
# Overview plots (grid)

if len(new_cols) > 0:

    set_renderer(renderer_map)
    fig, ax = plt.subplots(1, figsize=pdict["fsmap"])

    grid.loc[grid.incompatible_tags_sum == 0].plot(
        ax=ax, alpha=pdict["alpha_grid"], color=mpl.colors.rgb2hex(incompatible_false_patch.get_facecolor())
    )
    grid.loc[grid.incompatible_tags_sum > 0].plot(
        ax=ax, alpha=pdict["alpha_grid"], color=mpl.colors.rgb2hex(incompatible_true_patch.get_facecolor())
    )

    # add no data patches
    grid[grid["count_osm_edges"].isnull()].plot(
        ax=ax,
        facecolor=pdict["nodata_face"],
        edgecolor=pdict["nodata_edge"],
        linewidth= pdict["line_nodata"],
        hatch=pdict["nodata_hatch"],
        alpha=pdict["alpha_nodata"],
    )

    ax.legend(
        handles=[incompatible_true_patch, incompatible_false_patch, nodata_patch],
        loc="upper right",
    )

    ax.set_title(area_name+": OSM incompatible tag combinations")
    ax.set_axis_off()
    cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)

    plot_func.save_fig(fig, osm_results_static_maps_fp + "tagsincompatible_osm")

else:
    print("There are no incompatible tag combinations to plot.")

#### Plotting incompatible tag geometries

In [ ]:
incompatible_tags_edge_ids = eval_func.check_incompatible_tags(
    osm_edges, incompatible_tags_dict, store_edge_ids=True
)

if len(incompatible_tags_edge_ids) > 0:

    incompatible_tags_fg = []

    # iterate through dict of queries,
    for i, key in enumerate(list(incompatible_tags_edge_ids.keys())):
        # create one feature group for each query
        # and append it to list
        incompatible_tags_fg.append(
            plot_func.make_edgefeaturegroup(
                gdf=osm_edges[
                    osm_edges["edge_id"].isin(incompatible_tags_edge_ids[key])
                ],
                mycolor=pdict["basecols"][i],
                myweight=pdict["line_emp"],
                nametag="Incompatible tags: " + key,
                show_edges=True,
            )
        )

    ### Make marker feature group
    edge_ids = [
        item
        for sublist in list(incompatible_tags_edge_ids.values())
        for item in sublist
    ]  # get ids of all edges that have incompatible tags

    mfg = plot_func.make_markerfeaturegroup(
        osm_edges.loc[osm_edges["edge_id"].isin(edge_ids)], nametag="Incompatible tag marker", show_markers=True
    )
    incompatible_tags_fg.append(mfg)

    # create interactive map
    m = plot_func.make_foliumplot(
        feature_groups=incompatible_tags_fg,
        layers_dict=folium_layers,
        center_gdf=osm_nodes,
        center_crs=osm_nodes.crs,
    )

    bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
    m.fit_bounds(bounds)

    m.save(osm_results_inter_maps_fp + "tagsincompatible_osm.html")

    display(m)

In [ ]:
if len(incompatible_tags_edge_ids) > 0:
    print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + "tagsincompatible_osm.html")
else:
    print("There are no incompatible tag combinations to plot.")

### Tagging patterns

Identifying bicycle infrastructure in OSM can be tricky due to the many different ways in which the presence of bicycle infrastructure can be indicated. The [OSM Wiki](https://wiki.openstreetmap.org/wiki/Main_Page) is a great resource for recommendations for how OSM features should be tagged, but some inconsistencies and local variations can remain. The analysis of tagging patterns allows to visually explore some of the potential inconsistencies.

Regardless of how the bicycle infrastructure is defined, examining which tags contribute to which parts of the bicycle network allows to visually examine patterns in tagging methods. It also allows to estimate whether some elements of the query will lead to the inclusion of too many or too few features.

Likewise, 'double tagging' where several different tags have been used to indicate bicycle infrastructure can lead to misclassifications of the data. For this reason, identifying features that are included in more than one of the queries defining bicycle infrastructure can indicate issues with the tagging quality.

**Method**

We first plot individual subsets of the OSM data set for each of the queries listed in `bicycle_infrastructure_queries`, as defined in the `config.yml` file. The subset defined by a query is the set of edges for which this query is *True*. Since several queries can be *True* for the same edge, the subsets can overlap. In the second step below, all overlaps between 2 or more queries are plotted, i.e. all edges that have been assigned several, potentially competing, tags.

**Interpretation**

The plots for each tagging type allow for a quick visual overview of different tagging patterns present in the area. Based on local knowledge, the user may estimate whether the differences in tagging types are due to actual physical differences in the infrastructure or rather an artefact of the OSM data. Next, the user can access overlaps between different tags; depending on the specific tags, this may or may not be a data quality issue. For example, in case of `'cycleway:right'` and `'cycleway:left'`, having data for both tags is valid, but other combinations such as `'cycleway'='track'` and `'cycleway:left=lane'` gives an ambiguouos picture of what type of bicycle infrastructure is present.

#### Tagging types

In [ ]:
osm_edges["tagging_type"] = ""

for k, q in bicycle_infrastructure_queries.items():

    try:
        ox_filtered = osm_edges.query(q)

    except Exception:
        print("Exception occured when quering with:", q)

    osm_edges.loc[ox_filtered.index, "tagging_type"] = (
        osm_edges.loc[ox_filtered.index, "tagging_type"] + k
    )

In [ ]:
# Interactive plot of tagging types

# initialize list of folium feature groups (one per tagging type)
tagging_types_fg = []

# iterate through dict of queries,
i = 0  # index for color change
for key in bicycle_infrastructure_queries.keys():
    # create one feature group for each query
    # and append it to list
    tagging_types_fg.append(
        plot_func.make_edgefeaturegroup(
            gdf=osm_edges[osm_edges["tagging_type"] == key],
            mycolor=pdict["basecols"][i],
            myweight=2,
            nametag="Tagging type " + key,
            show_edges=True,
        )
    )
    i += 1  # update index

# create interactive map
m = plot_func.make_foliumplot(
    feature_groups=tagging_types_fg,
    layers_dict=folium_layers,
    center_gdf=osm_nodes_simplified,
    center_crs=osm_nodes_simplified.crs,
)

bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
m.fit_bounds(bounds)

m.save(osm_results_inter_maps_fp + "taggingtypes_osm.html")

for k, v in bicycle_infrastructure_queries.items():
    print("Tagging type " + k + ": " + v)

display(m)

In [ ]:
print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + "taggingtypes_osm.html")

#### Multiple tagging

In [ ]:
# Plot all osm_edges which are returned for more than one tagging_type

set_renderer(renderer_map)
tagging_combinations = list(osm_edges.tagging_type.unique())
tagging_combinations = [x for x in tagging_combinations if len(x) > 1]

if len(tagging_combinations) > 0:

    for i, t in enumerate(tagging_combinations):

        fig, ax = plt.subplots(1, figsize=pdict["fsmap"])

        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if len(osm_edges.loc[osm_edges["tagging_type"] == t]) < 10:

            grid.plot(ax=ax, facecolor="none", edgecolor="none", alpha=0)
            osm_edges.loc[osm_edges["tagging_type"] == t]["geometry"].centroid.plot(
                ax=ax, color=pdict["osm_emp"], markersize=30
            )

        else:
            grid.plot(ax=ax, facecolor="none", edgecolor="none", alpha=0)
            osm_edges.loc[osm_edges["tagging_type"] == t]["geometry"].centroid.plot(
                ax=ax, color=pdict["osm_emp"], markersize=5
            )


        columns_in_query = ""
        for value in t:
            tag_name = bicycle_infrastructure_queries[value].split(" ")[0]
            columns_in_query = columns_in_query + tag_name + " + "

        columns_in_query = columns_in_query[:-3]

        title = area_name+": OSM bicycle infrastructure defined with tags: " + columns_in_query

        ax.set_title(title)

        cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)

        filepath_extension = columns_in_query.replace(" + ", "-")
        
        plot_func.save_fig(fig, osm_results_static_maps_fp + f"tagscompeting_{filepath_extension}_osm")

    plt.show()

In [ ]:
if len(tagging_combinations) > 0:

    # set seed for colors
    np.random.seed(42)

    # generate enough random colors to plot all tagging combinations
    randcols = [colors.to_hex(rgbcol) for rgbcol in np.random.rand(len(tagging_combinations), 3)]

    # initialize list of feature groups (one for each tagging combination)
    tag_featuregroups = []
    
    for i, t in enumerate(tagging_combinations):

        columns_in_query = ""

        for value in t:
            tag_name = bicycle_infrastructure_queries[value].split(" ")[0]
            columns_in_query = columns_in_query + tag_name + " + "

        columns_in_query = columns_in_query[:-3]

        tag_featuregroup = plot_func.make_edgefeaturegroup(
            gdf=osm_edges.loc[osm_edges["tagging_type"] == t],
            mycolor=randcols[i],
            myweight=pdict["line_emp"],
            nametag=columns_in_query,
            show_edges=True,
        )

        tag_featuregroups.append(tag_featuregroup)

        
    m = plot_func.make_foliumplot(
        feature_groups=tag_featuregroups,
        layers_dict=folium_layers,
        center_gdf=osm_nodes_simplified,
        center_crs=osm_nodes_simplified.crs,
    )

    bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
    m.fit_bounds(bounds)

    m.save(osm_results_inter_maps_fp + "taggingcombinations_osm.html") 

    display(m)

In [ ]:
if len(tagging_combinations) > 0:
    print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + "taggingcombinations_osm.html")
else:
    print("There are no tagging combinations to plot.")

## Network topology

This section explores the geometric and topological features of the data. These are, for example, network density, disconnected components, and dangling (degree one) nodes. It also includes exploring whether there are nodes that are very close to each other but do not share an edge - a potential sign of edge undershoots - or if there are intersecting edges without a node at the intersection, which might indicate a digitizing error that will distort routing on the network.

Due to the fragmented nature of most bicycle networks, many metrics, such as missing links or network gaps, can simply reflect the true extent of the infrastructure ([Natera Orozco et al., 2020](https://onlinelibrary.wiley.com/doi/pdfdirect/10.1111/gean.12324)). This is different for road networks, where e.g., disconnected components could more readily be interpreted as a data quality issue. Therefore, the analysis only takes very small network gaps into account as potential data quality issues.

### Simplification outcome

To compare the structure and true ratio between nodes and edges in the network, a simplified network representation which only includes nodes at endpoints and intersections was created in notebook `1a` by removing all interstitial nodes.

Comparing the degree distribution for the networks before and after simplification is a quick sanity check for the simplification routine. Typically, the vast majority of nodes in the non-simplified network will be of degree two; in the simplified network, however, most nodes will have degrees other than two. Degree two nodes are retained in only two cases: if they represent a connection point between two different types of infrastructure; or if they are needed in order to avoid self-loops (edges whose start and end points are identical) or multiple edges between the same pair of nodes. 

<p align="center">
<img src='../../images/network_simplification_illustration.png' width=300/>

*Non-simplified network (left) and simplified network (right)*.

</p>

**Method**

The degree distributions before and after simplification are plotted below. 

**Interpretation**

Typically, the degree distribution will go from high (before simplification) to low (after simplification) counts of degree two nodes, while it will not change for all other degrees (1, or 3 and higher). Further, the total number of nodes will see a strong decline. If the simplified graph still maintains a relatively high number of degree two nodes, or if the number of nodes with other degrees changes after the simplification, this might point to issues either with the graph conversion or with the simplification process.

In [ ]:
# Decrease in network elements after simplification

edge_percent_diff = (len(osm_edges) - len(osm_edges_simplified)) / len(osm_edges) * 100
node_percent_diff = (len(osm_nodes) - len(osm_nodes_simplified)) / len(osm_nodes) * 100

simplification_results = {
    "edge_percent_diff": edge_percent_diff,
    "node_percent_diff": node_percent_diff,
}

print(
    f"Simplifying the network decreased the number of edges by {edge_percent_diff:.1f}% and the number of nodes by {node_percent_diff:.1f}%."
)

In [ ]:
# Degree distribution

set_renderer(renderer_plot)
fig, ax = plt.subplots(1, 2, figsize=pdict["fsbar_short"], sharey=True)

degree_sequence_before = sorted((d for n, d in osm_graph.degree()), reverse=True)
degree_sequence_after = sorted(
    (d for n, d in osm_graph_simplified.degree()), reverse=True
)

# Plot degree distributions
ax[0].bar(*np.unique(degree_sequence_before, return_counts=True), tick_label = np.unique(degree_sequence_before), color=pdict["osm_base"])
ax[0].set_title("Before simplification")
ax[0].set_xlabel("Degree")
ax[0].set_ylabel("Nodes")

ax[1].bar(*np.unique(degree_sequence_after, return_counts=True), tick_label = np.unique(degree_sequence_after), color=pdict["osm_base"])
ax[1].set_title("After simplification")
ax[1].set_xlabel("Degree")

plt.suptitle(f"{area_name}: OSM degree distributions")

fig.tight_layout()

plot_func.save_fig(fig, osm_results_plots_fp + "degree_dist_osm")

plt.show();

### Dangling nodes

Dangling nodes are nodes of degree one, i.e. they have only one single edge attached to them. Most networks will naturally contain a number of dangling nodes. Dangling nodes can occur at actual dead-ends (representing a cul-de-sac) or at the endpoints of certain features, e.g. when a bicycle path ends in the middle of a street. However, dangling nodes can also occur as a data quality issue in case of over/undershoots (see next section). The number of dangling nodes in a network does to some extent also depend on the digitization method, as shown in the illustration below. 

Therefore, the presence of dangling nodes is in itself not a sign of low data quality. However, a high number of dangling nodes in an area that is not known for containing many dead-ends can indicate digitization errors and problems with edge over/undershoots.

<!-- <table><tr><td><img src='../../images/dangling_nodes_illustration.png' width=300 /></td><td><img src='../../images/no_dangling_nodes_illustration.png' width=295 /></td></tr></table>

*Left: Dangling nodes occur where road features end. Right: However, when separate features are joined at the end, there will be no dangling nodes.* -->

<p align="center">

<img src='../../images/dangling_nodes_illustration_new.png' width=350/>

*Left: Dangling nodes occur where road features end. Right: However, when separate features are joined at the end, there will be no dangling nodes.*

</p>

**Method**

Below, a list of all dangling nodes is obtained with the help of `get_dangling_nodes`. Then, the network with all its nodes is plotted. The dangling nodes are shown in color, all other nodes are shown in black.

**Interpretation**

We recommend a visual analysis in order to interpret the spatial distribution of dangling nodes, with particular attention to areas of high dangling node density. It is important to understand where dangling nodes come from: are they actual dead-ends or digitization errors (e.g., over/undershoots)? A higher number of digitization errors points to lower data quality.

<br />

In [ ]:
# Compute number of dangling nodes
dangling_nodes = eval_func.get_dangling_nodes(
    osm_edges_simplified, osm_nodes_simplified
)

# Export results
dangling_nodes.to_file(osm_results_data_fp + "dangling_nodes.gpkg", index=False)

# Compute local count and pct of dangling nodes
dn_osm_joined = gpd.overlay(
    dangling_nodes, grid[["geometry", "grid_id"]], how="intersection"
)
df = eval_func.count_features_in_grid(dn_osm_joined, "osm_dangling_nodes")
grid = eval_func.merge_results(grid, df, "left")

grid["osm_dangling_nodes_pct"] = np.round(
    100 * grid.count_osm_dangling_nodes / grid.count_osm_simplified_nodes, 2
)

# set to zero where there are simplified nodes but no dangling nodes
grid["osm_dangling_nodes_pct"].loc[
    grid.count_osm_simplified_nodes.notnull() & grid.osm_dangling_nodes_pct.isnull()
] = 0

In [ ]:
# Plot dangling nodes

set_renderer(renderer_map)
fig, ax = plt.subplots(1, figsize=pdict["fsmap"])

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3.5%", pad="1%")

grid.plot(
    cax=cax,
    column="osm_dangling_nodes_pct",
    ax=ax,
    alpha=pdict["alpha_grid"],
    cmap=pdict["pos"],
    legend=True,
)

# add no data patches
grid[grid["count_osm_simplified_nodes"].isnull()].plot(
    cax=cax,
    ax=ax,
    facecolor=pdict["nodata_face"],
    edgecolor=pdict["nodata_edge"],
    linewidth= pdict["line_nodata"],
    hatch=pdict["nodata_hatch"],
    alpha=pdict["alpha_nodata"],
)

ax.legend(handles=[nodata_patch], loc="upper right")
ax.set_title(f"{area_name}: OSM percent of dangling nodes")
ax.set_axis_off()
cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)

plot_func.save_fig(fig, osm_results_static_maps_fp + "pct_dangling_nodes_osm")

In [ ]:
# Interactive plot of dangling nodes

edges_simplified_folium = plot_func.make_edgefeaturegroup(
    gdf=osm_edges_simplified,
    mycolor=pdict["base"],
    myweight=pdict["line_base"],
    nametag="Edges",
    show_edges=True,
)

nodes_simplified_folium = plot_func.make_nodefeaturegroup(
    gdf=osm_nodes_simplified,
    mysize=pdict["mark_base"],
    mycolor=pdict["base"],
    nametag="All nodes",
    show_nodes=True,
)

dangling_nodes_folium = plot_func.make_nodefeaturegroup(
    gdf=dangling_nodes,
    mysize=pdict["mark_emp"],
    mycolor= pdict["osm_base"],
    nametag="Dangling nodes",
    show_nodes=True,
)

m = plot_func.make_foliumplot(
    feature_groups=[
        edges_simplified_folium,
        nodes_simplified_folium,
        dangling_nodes_folium,
    ],
    layers_dict=folium_layers,
    center_gdf=osm_nodes_simplified,
    center_crs=osm_nodes_simplified.crs,
)

bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
m.fit_bounds(bounds)

m.save(osm_results_inter_maps_fp + "danglingmap_osm.html")

display(m)

In [ ]:
print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + "danglingmap_osm.html")

### Under/overshoots 

When two nodes in a simplified network are placed within a distance of a few meters, but do not share a common edge, it is often due to an edge over/undershoot or another digitizing error. An undershoot occurs when two features are supposed to meet, but instead are just in close proximity to each other. An overshoot occurs when two features meet and one of them extends beyond the other. See the image below for an illustration. For a more detailed explanation of over/undershoots, see the [GIS Lounge website](https://www.gislounge.com/digitizing-errors-in-gis/).

<p align="center">

<img src='../../images/over_undershoots2.png' width=350/>

*Left: Undershoots happen when two line features are not properly joined, for example at an intersection. Right: Overshoots refer to situations where a line feature extends too far beyond at intersecting line, rather than ending at the intersection.* 

</p>


**Method**

*Undershoots:* First, the `length_tolerance` (in meters) is defined in the cell below. Then, with `find_undershoots`, all pairs of dangling nodes that have a maximum of `length_tolerance` distance between them, are identified as undershoots, and the results are plotted.

*Overshoots:* First, the `length_tolerance` (in meters) is defined in the cell below. Then, with `find_overshoots`, all network edges that have a dangling node attached to them and that have a maximum length of `length_tolerance` are identifed as overshoots, and the results are plotted.

The method for over/undershoot detection is inspired by [Neis et al. (2012)](https://www.mdpi.com/1999-5903/4/1/1).

**Interpretation**

Under/overshoots are not necessarily always a data quality issue - they might be instead an accurate representation of the network conditions or of the digitization strategy. For example, a cycle path might end abruptly soon after a turn, which results in an overshoot. Protected cycle paths are sometimes digitized in OSM as interrupted at intersections which results in intersection undershoots.

The interpretation of the impact of over/undershoots on data quality is context dependent. For certain applications, such as routing, overshoots do not present a particular challenge; they can, however, pose an issue for other applications such as network analysis, given that they skew the network structure.  Undershoots, on the contrary, are a serious problem for routing applications, especially if only bicycle infrastructure is considered. They also pose a problem for network analysis, for example for any path-based metric, such as most centrality measures like betweenness centrality.

<br />

<div class="alert alert-block alert-info">
<b>User configurations</b>
<br>
<br>
In the analysis of over and undershoots, the user can modify the length tolerance for both over and undershoots. <br>
For example, a length tolerance of 3 meters for overshoots means that only edge snippets with a length of 3 meters or less are considered overshoots.<br>
A tolerance of 5 meters for undershoots means that only gaps of 5 meters or less are considered undershoots.
</div>

In [ ]:
# USER INPUT: LENGTH TOLERANCE FOR OVER- AND UNDERSHOOTS
length_tolerance_over = 3
length_tolerance_under = 3

for s in [length_tolerance_over, length_tolerance_under]:
    assert isinstance(s, int) or isinstance(s, float), print(
        "Settings must be integer or float values!"
    )

print(f"Running overshoot analysis with a tolerance threshold of {length_tolerance_over} m.")
print(f"Running undershoot analysis with a tolerance threshold of {length_tolerance_under} m.")

In [ ]:
### Overshoots

overshoots = eval_func.find_overshoots(
    dangling_nodes,
    osm_edges_simplified,
    length_tolerance_over,
    return_overshoot_edges=True,
)

print(
    f"{len(overshoots)} potential overshoots were identified using a length tolerance of {length_tolerance_over} m."
)

### Undershoots
undershoot_dict, undershoot_nodes = eval_func.find_undershoots(
    dangling_nodes,
    osm_edges_simplified,
    length_tolerance_under,
    "edge_id",
    return_undershoot_nodes=True,
)

print(
    f"{len(undershoot_nodes)} potential undershoots were identified using a length tolerance of {length_tolerance_under} m."
)

In [ ]:
# Save to csv

overshoots[["edge_id", "length"]].to_csv(
    osm_results_data_fp + f"overshoot_edges_{length_tolerance_over}.csv", header = ["edge_id", "length (m)"], index = False
)

pd.DataFrame(undershoot_nodes["osmid"].to_list(), columns=["node_id"]).to_csv(
    osm_results_data_fp + f"undershoot_nodes_{length_tolerance_under}.csv", index=False
)

In [ ]:
# Interactive plot of under/overshoots

simplified_edges_folium = plot_func.make_edgefeaturegroup(
    gdf=osm_edges_simplified,
    mycolor=pdict["base"],
    myweight=pdict["line_base"],
    nametag="Edges",
    show_edges=True,
)

fg = [simplified_edges_folium]

if len(overshoots) > 0 or len(undershoot_nodes) > 0:

    if len(overshoots) > 0:

        overshoots_folium = plot_func.make_edgefeaturegroup(
            gdf=overshoots,
            mycolor=pdict["osm_contrast"],
            myweight=pdict["line_emp2"],
            nametag="Overshoots",
            show_edges=True,
        )

        fg.append(overshoots_folium)

    if len(undershoot_nodes) > 0:

        undershoot_nodes_folium = plot_func.make_nodefeaturegroup(
            gdf=undershoot_nodes,
            mysize=pdict["mark_emp"],
            mycolor=pdict["osm_contrast2"],
            nametag="Undershoot nodes",
            show_nodes=True,
        )

        fg.append(undershoot_nodes_folium)

    m = plot_func.make_foliumplot(
        feature_groups=fg,
        layers_dict=folium_layers,
        center_gdf=osm_nodes_simplified,
        center_crs=osm_nodes_simplified.crs,
    )

    bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
    m.fit_bounds(bounds)

    m.save(
        osm_results_inter_maps_fp
        + f"underovershoots_{length_tolerance_under}_{length_tolerance_over}_osm.html"
    )

    display(m)

if len(undershoot_nodes) == 0:
    print("There are no undershoots to plot.")
if len(overshoots) == 0:
    print("There are no overshoots to plot.")

In [ ]:
if len(overshoots) > 0 or len(undershoot_nodes) > 0:
    print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + f"underovershoots_{length_tolerance_under}_{length_tolerance_over}_osm.html")
else:
    print("There are no under/overshoots to plot.")

### Missing intersection nodes

When two edges intersect without having a node at the intersection - and if neither edges are tagged as a bridge or a tunnel - there is a clear indication of a topology error. 

**Method**

First, with the help of `check_intersection`, each edge which is not tagged as either tunnel or bridge is checked for any *crossing* with another edge of the network. If this is the case, the edge is marked as having an intersection issue. The number of intersection issues found is printed and the results are plotted for visual analysis. The method is inspired by [Neis et al. (2012)](https://www.mdpi.com/1999-5903/4/1/1).

**Interpretation**

A higher number of intersection issues points to a lower data quality. However, it is recommended with a manual visual check of all intersection issues with a certain knowledge of the area, in order to determine the origin of intersection issues and confirm/correct/reject them.

<div class="alert alert-block alert-danger">
<b>Warning</b>
<p>
This is the most computationally intensive operation in this notebook. It can take several times longer than all other sections.
</p>
</div>

In [ ]:
missing_nodes_edge_ids, edges_with_missing_nodes = eval_func.find_missing_intersections(
    osm_edges, "edge_id"
)

count_intersection_issues = (
    len(missing_nodes_edge_ids) / 2
)  # The number of issues is counted twice since both intersecting osm_edges are returned

print(
    f"{count_intersection_issues:.0f} place(s) appear to be missing an intersection node or a bridge/tunnel tag."
)

In [ ]:
# Save to csv

if count_intersection_issues > 0: 
    pd.DataFrame(data=missing_nodes_edge_ids, columns=["edge_id"]).to_csv(
        osm_results_data_fp + "edges_missing_intersections.csv", index=False
    )

In [ ]:
# Interactive plot of intersection issues

if count_intersection_issues > 0:

    simplified_edges_folium = plot_func.make_edgefeaturegroup(
        gdf=osm_edges_simplified,
        mycolor=pdict["base"],
        myweight=pdict["line_base"],
        nametag="All edges",
        show_edges=True,
    )

    intersection_issues_folium = plot_func.make_edgefeaturegroup(
        gdf=edges_with_missing_nodes,
        mycolor=pdict["osm_contrast"],
        myweight=pdict["line_emp"],
        nametag="Intersection issues: edges",
        show_edges=True,
    )

    mfg = plot_func.make_markerfeaturegroup(
        edges_with_missing_nodes, 
        nametag="Intersection issues: marker at missing node", 
        show_markers=True
    )
  
    m = plot_func.make_foliumplot(
        feature_groups=[simplified_edges_folium, intersection_issues_folium, mfg],
        layers_dict=folium_layers,
        center_gdf=osm_nodes_simplified,
        center_crs=osm_nodes_simplified.crs,
    )

    bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
    m.fit_bounds(bounds)

    m.save(osm_results_inter_maps_fp + "intersection_issues_osm.html")

    display(m)

In [ ]:
if count_intersection_issues > 0:
    print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + "intersection_issues_osm.html")
else:
    print("There are no intersection problems to plot.")

## Network components

Disconnected components do not share any elements (nodes/edges). In other words, there is no network path that could lead from one disconnected component to the other. As mentioned above, most real-world networks of bicycle infrastructure do consist of many disconnected components ([Natera Orozco et al., 2020](https://onlinelibrary.wiley.com/doi/pdfdirect/10.1111/gean.12324)). However, when two disconnected components are very close to each other, it might be a sign of a missing edge or another digitizing error.

**Method**

First, with the help of `return_components`, a list of all (disconnected) components of the network is obtained. The total number of components is printed and all components are plotted in different colors for visual analysis. Next, the component size distribution (with components ordered by the network length they contain) is plotted, followed by a plot of the largest connected component. 

**Interpretation**

As with many of the previous analysis steps, knowledge of the area is crucial for a correct interpretation of component analysis. Given that the data represents the actual infrastructure accurately, bigger components indicate coherent network parts, while smaller components indicate scattered infrastructure (e.g., one single bicycle path along a street that does not connect to any other bicycle infrastructure). A high number of disconnected components in near vicinity of each other indicates digitization errors or missing data.

### Disconnected components

In [ ]:
osm_components = eval_func.return_components(osm_graph_simplified)
print(
    f"The network in the study area has {len(osm_components)} disconnected components."
)

In [ ]:
# Plot disconnected components

set_renderer(renderer_map)

# set seed for colors
np.random.seed(42)

# generate enough random colors to plot all components
randcols = np.random.rand(len(osm_components), 3)
randcols[0, :] = col_to_rgb(pdict['osm_base'])

fig, ax = plt.subplots(1, 1, figsize=pdict["fsmap"])

ax.set_title(f"{area_name}: OSM disconnected components")

ax.set_axis_off()

for j, c in enumerate(osm_components):
    if len(c.edges) > 0:
        edges = ox.graph_to_gdfs(c, nodes=False)
        edges.plot(ax=ax, color=randcols[j])

cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)

plot_func.save_fig(fig, osm_results_static_maps_fp + "all_components_osm")

### Components per grid cell

In [ ]:
# Assign component ids to grid

grid = eval_func.assign_component_id_to_grid(
    osm_edges_simplified,
    osm_edges_simp_joined,
    osm_components,
    grid,
    prefix="osm",
    edge_id_col="edge_id",
)

fill_na_dict = {"component_ids_osm": ""}
grid.fillna(value=fill_na_dict, inplace=True)

grid["component_count_osm"] = grid.component_ids_osm.apply(lambda x: len(x))


In [ ]:
# Plot number of components per grid cell

set_renderer(renderer_map)

fig, ax = plt.subplots(1, 1, figsize=pdict["fsmap"])

ncolors = grid["component_count_osm"].max()

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3.5%", pad="1%")

mycm = cm.get_cmap(pdict["seq"], ncolors) 
grid[grid.component_count_osm>0].plot(
    cax=cax,
    ax=ax,
    column="component_count_osm",
    legend=True,
    legend_kwds={'ticks': list(range(1, ncolors+1))},
    cmap=mycm,
    alpha=pdict["alpha_grid"],
)

# add no data patches
grid[grid["count_osm_edges"].isnull()].plot(
    cax=cax,
    ax=ax,
    facecolor=pdict["nodata_face"],
    edgecolor=pdict["nodata_edge"],
    linewidth= pdict["line_nodata"],
    hatch=pdict["nodata_hatch"],
    alpha=pdict["alpha_nodata"],
)

ax.legend(handles=[nodata_patch], loc="upper right")

cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)
ax.set_title(area_name + ": OSM number of components in grid cells")
ax.set_axis_off()

plot_func.save_fig(fig, osm_results_static_maps_fp + f"number_of_components_in_grid_cells_osm")

### Component length distribution

The distribution of all network component lengths can be visualized in a so-called *Zipf plot*, which orders the lengths of each component by rank, showing the largest component's length on the left, then the second largest component's length, etc., until the smallest component's length on the right. When a Zipf plot follows a straight line in [log-log scale](https://en.wikipedia.org/wiki/Logarithmic_scale), it means that there is a much higher chance to find small disconnected components than expected from traditional distributions [(Clauset et al., 2009)]( https://epubs.siam.org/doi/abs/10.1137/070710111). This can mean that there has been no consolidation of the network, only piece-wise or random additions [(Szell et al., 2022)](https://www.nature.com/articles/s41598-022-10783-y), or that the data itself suffers from many gaps and topology errors resulting in small disconnected components.

However, it can also happen that the largest connected component (the leftmost marker in the plot at rank $10^0$) is a clear outlier, while the rest of the plot follows a different shape. This can mean that at the infrastructure level, most of the infrastructure has been connected to one large component, and that the data reflects this - i.e. the data is not suffering from gaps and missing links to a large extent.

Bicycle networks might also be somewhere inbetween, with several large components as outliers.

In [ ]:
# Zipf plot of component lengths

set_renderer(renderer_plot)

components_length = {}
for i, c in enumerate(osm_components):
    c_length = 0
    for (u, v, l) in c.edges(data="length"):
        c_length += l
    components_length[i] = c_length

components_df = pd.DataFrame.from_dict(components_length, orient="index")
components_df.rename(columns={0: "component_length"}, inplace=True)

fig = plt.figure(figsize=pdict["fsbar_small"])
axes = fig.add_axes([0, 0, 1, 1])

axes.set_axisbelow(True)
axes.grid(True,which="major",ls="dotted")
yvals = sorted(list(components_df["component_length"] / 1000), reverse = True)
axes.scatter(
    x=[i+1 for i in range(len(components_df))],
    y=yvals,
    s=18,
    color=pdict["osm_base"],
)
axes.set_ylim(ymin=10**math.floor(math.log10(min(yvals))), ymax=10**math.ceil(math.log10(max(yvals))))
axes.set_xscale("log")
axes.set_yscale("log")

axes.set_ylabel("Component length [km]")
axes.set_xlabel("Component rank (largest to smallest)")
axes.set_title(area_name+": OSM component length distribution")

plot_func.save_fig(fig, osm_results_plots_fp + "component_length_distribution_osm")

### Largest connected component

In [ ]:
largest_cc = max(osm_components, key=len)

largest_cc_length = 0

for (u, v, l) in largest_cc.edges(data="length"):

    largest_cc_length += l

largest_cc_pct = largest_cc_length / components_df["component_length"].sum() * 100

print(
    f"The largest connected component contains {largest_cc_pct:.2f}% of the network length."
)

# Get edges in largest cc
lcc_edges = ox.graph_to_gdfs(
    G=largest_cc, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=False
)

# Export to GPKG
lcc_edges[["edge_id", "geometry"]].to_file(
    osm_results_data_fp + "largest_connected_component.gpkg"
)

In [ ]:
# Plot of largest connected component

set_renderer(renderer_map)
fig, ax = plt.subplots(1, 1, figsize=pdict["fsmap"])
osm_edges_simplified.plot(ax=ax, color = pdict["base"], linewidth = 1.5, label = "All smaller components")
lcc_edges.plot(ax=ax, color=pdict["osm_base"], linewidth = 2, label = "Largest connected component")
grid.plot(ax=ax,alpha=0)
ax.set_axis_off()
ax.set_title(area_name + ": OSM largest connected component")
ax.legend()

cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)

plot_func.save_fig(fig, osm_results_static_maps_fp + f"largest_conn_comp_osm")

In [ ]:
# Save plot without basemap for potential report titlepage

set_renderer(renderer_map)
fig, ax = plt.subplots(1, 1, figsize=pdict["fsmap"])
osm_edges_simplified.plot(ax=ax, color = pdict["base"], linewidth = 1.5, label = "Disconnected components")
lcc_edges.plot(ax=ax, color=pdict["osm_base"], linewidth = 2, label = "Largest connected component")
ax.set_axis_off()

plot_func.save_fig(fig, osm_results_static_maps_fp + f"titleimage",plot_res="high")
plt.close()

### Missing links

In the plot of potential missing links between components, all edges that are within the specified distance of an edge on another component are plotted. The gaps between disconnected edges are highlighted with a marker. The map thus highlights edges which, despite being in close proximity of each other, are disconnected and where it thus would not be possible to bike on cycling infrastructure between the edges.

<div class="alert alert-block alert-info">
<b>User configuration</b>
<p>
In the analysis of potential missing links between components, the user must define the threshold for when the distance between two components is considered to be low enough that a digitization error is suspected.
    </p>
</div>

In [ ]:
# DEFINE MAX BUFFER DISTANCE BETWEEN COMPONENTS CONSIDERED A GAP/MISSING LINK
component_min_distance = 10

assert isinstance(component_min_distance, int) or isinstance(
    component_min_distance, float
), print("Setting must be integer or float value!")

print(f"Running analysis with component distance threshold of {component_min_distance} meters.")

In [ ]:
component_gaps = eval_func.find_adjacent_components(
    components=osm_components,
    buffer_dist=component_min_distance,
    crs=study_crs,
    edge_id="edge_id",
)
component_gaps_gdf = gpd.GeoDataFrame.from_dict(
    component_gaps, orient="index", geometry="geometry", crs=study_crs
)

edge_ids = set(
    component_gaps_gdf["edge_id" + "_left"].to_list()
    + component_gaps_gdf["edge_id" + "_right"].to_list()
)

edge_ids = [int(i) for i in edge_ids]
edges_with_gaps = osm_edges_simplified.loc[osm_edges_simplified.edge_id.isin(edge_ids)]

In [ ]:
# Save to csv
pd.DataFrame(edge_ids, columns=["edge_id"]).to_csv(
    osm_results_data_fp + f"component_gaps_edges_{component_min_distance}.csv",
    index=False,
)

# Export gaps to GPKG
component_gaps_gdf.to_file(
    osm_results_data_fp + f"component_gaps_centroids_{component_min_distance}.gpkg"
)

In [ ]:
# Interactive plot of adjacent, potentially disconnected components

if len(component_gaps) > 0:

    simplified_edges_folium = plot_func.make_edgefeaturegroup(
        gdf=osm_edges_simplified,
        mycolor=pdict["osm_base"],
        myweight=pdict["line_base"],
        nametag="All edges",
        show_edges=True,
    )

    component_issues_edges_folium = plot_func.make_edgefeaturegroup(
        gdf=edges_with_gaps,
        mycolor=pdict["osm_emp"],
        myweight=pdict["line_emp"],
        nametag="Adjacent disconnected edges",
        show_edges=True,
    )

    component_issues_gaps_folium = plot_func.make_markerfeaturegroup(
        gdf=component_gaps_gdf, nametag="Component gaps", show_markers=True
    )

    m = plot_func.make_foliumplot(
        feature_groups=[
            simplified_edges_folium,
            component_issues_edges_folium,
            component_issues_gaps_folium,
        ],
        layers_dict=folium_layers,
        center_gdf=osm_nodes_simplified,
        center_crs=osm_nodes_simplified.crs,
    )

    bounds = plot_func.compute_folium_bounds(osm_nodes_simplified)
    m.fit_bounds(bounds)

    m.save(osm_results_inter_maps_fp + f"component_gaps_{component_min_distance}_osm.html")

    display(m)

In [ ]:
if len(component_gaps) > 0:
    print("Interactive map saved at " + osm_results_inter_maps_fp.lstrip("../") + f"component_gaps_{component_min_distance}_osm.html")
else:
    print("There are no component gaps to plot.")

### Component connectivity

Here we visualize differences between how many cells can be reached from each cell. This is a crude measure for network connectivity but has the benefit of being computationally cheap and thus able to quickly highlight stark differences in network connectivity.

In [ ]:
osm_components_cell_count = eval_func.count_component_cell_reach(
    components_df, grid, "component_ids_osm"
)
grid["cells_reached_osm"] = grid["component_ids_osm"].apply(
    lambda x: eval_func.count_cells_reached(x, osm_components_cell_count)
    if x != ""
    else 0
)

grid["cells_reached_osm_pct"] = grid.apply(
    lambda x: np.round((x.cells_reached_osm / len(grid)) * 100, 2), axis=1
)

grid.loc[grid["cells_reached_osm_pct"] == 0, "cells_reached_osm_pct"] = np.NAN

In [ ]:
# Plot percent of cells reachable

set_renderer(renderer_map)
fig, ax = plt.subplots(1, 1, figsize=pdict["fsmap"])

# norm for color bars
cbnorm_reach = colors.Normalize(vmin=0, vmax=100)

from mpl_toolkits.axes_grid1 import make_axes_locatable
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3.5%", pad="1%")

grid[grid.cells_reached_osm_pct > 0].plot(
    cax=cax,
    ax=ax,
    column="cells_reached_osm_pct",
    legend=True,
    cmap=pdict["seq"],
    norm=cbnorm_reach,
    alpha=pdict["alpha_grid"],
)

osm_edges_simplified.plot(ax=ax, color=pdict["osm_emp"], linewidth=1)

# add no data patches
grid[grid["count_osm_edges"].isnull()].plot(
    cax=cax,
    ax=ax,
    facecolor=pdict["nodata_face"],
    edgecolor=pdict["nodata_edge"],
    linewidth= pdict["line_nodata"],
    hatch=pdict["nodata_hatch"],
    alpha=pdict["alpha_nodata"],
)

ax.legend(handles=[nodata_patch], loc="upper right")

cx.add_basemap(ax=ax, crs=study_crs, source=cx_tile_2)
ax.set_title(area_name+": OSM percent of cells reachable")
ax.set_axis_off()

plot_func.save_fig(fig, osm_results_static_maps_fp + "percent_cells_reachable_grid_osm")

In [ ]:
components_results = {}
components_results["component_count"] = len(osm_components)
components_results["largest_cc_pct_size"] = largest_cc_pct
components_results["largest_cc_length"] = largest_cc_length
components_results["count_component_gaps"] = len(component_gaps)

## Summary

In [ ]:
# Print out table summary of results

summarize_results = {**density_results, **components_results}

summarize_results["count_dangling_nodes"] = len(dangling_nodes)
summarize_results["count_intersection_issues"] = count_intersection_issues
summarize_results["count_overshoots"] = len(overshoots)
summarize_results["count_undershoots"] = len(undershoot_nodes)
summarize_results["count_incompatible_tags"] = sum(
    len(lst) for lst in incompatible_tags_results.values()
)

# Add total node count and total infrastructure length
summarize_results["total_nodes"] = len(osm_nodes_simplified)
summarize_results["total_length"] = osm_edges_simplified.infrastructure_length.sum() / 1000

summarize_results_df = pd.DataFrame.from_dict(summarize_results, orient="index")

summarize_results_df.rename({0: " "}, axis=1, inplace=True)

# Convert length to km
summarize_results_df.loc["largest_cc_length"] = (
    summarize_results_df.loc["largest_cc_length"] / 1000
)

summarize_results_df = summarize_results_df.reindex([
    'total_length',
    'protected_density_m_sqkm',
    'unprotected_density_m_sqkm',
    'mixed_density_m_sqkm',
    'edge_density_m_sqkm',
    'total_nodes',
    'count_dangling_nodes',
    'node_density_count_sqkm',
    'dangling_node_density_count_sqkm',
    'count_incompatible_tags',
    'count_overshoots',
    'count_undershoots',
    'count_intersection_issues',
    'component_count',
    'largest_cc_length',
    'largest_cc_pct_size', 
    'count_component_gaps'
     ])

rename_metrics = {
    "total_length": "Total infrastructure length (km)",
    "total_nodes": "Nodes",
    "edge_density_m_sqkm": "Bicycle infrastructure density (m/km2)",
    "node_density_count_sqkm": "Nodes per km2",
    "dangling_node_density_count_sqkm": "Dangling nodes per km2",
    "protected_density_m_sqkm": "Protected bicycle infrastructure density (m/km2)",
    "unprotected_density_m_sqkm": "Unprotected bicycle infrastructure density (m/km2)",
    "mixed_density_m_sqkm": "Mixed protection bicycle infrastructure density (m/km2)",
    "component_count": "Components",
    "largest_cc_pct_size": "Largest component's share of network length",
    "largest_cc_length": "Length of largest component (km)",
    "count_component_gaps": "Component gaps",
    "count_dangling_nodes": "Dangling nodes",
    "count_intersection_issues": "Missing intersection nodes",
    "count_overshoots": "Overshoots",
    "count_undershoots": "Undershoots",
    "count_incompatible_tags": "Incompatible tag combinations",
}

summarize_results_df.rename(rename_metrics, inplace=True)
summarize_results_df.style.pipe(format_osm_style)

## Save results

In [ ]:
all_results = {}

all_results["existing_tags"] = existing_tags_results
all_results["incompatible_tags_results"] = incompatible_tags_results
all_results["incompatible_tags_count"] = sum(
    len(lst) for lst in incompatible_tags_results.values()
)
all_results["network_density"] = density_results
all_results["count_intersection_issues"] = count_intersection_issues
all_results["count_overshoots"] = len(overshoots)
all_results["count_undershoots"] = len(undershoot_nodes)
all_results["dangling_node_count"] = len(dangling_nodes)
all_results["simplification_outcome"] = simplification_results
all_results["component_analysis"] = components_results

with open(osm_intrinsic_fp, "w") as outfile:
    json.dump(all_results, outfile)


# Save summary dataframe
summarize_results_df.to_csv(
    osm_results_data_fp + "intrinsic_summary_results.csv", index=True
)

# Save grid with results
with open(osm_intrinsic_grid_fp, "wb") as f:
    pickle.dump(grid, f)

***

In [ ]:
from time import strftime
print("Time of analysis: " + strftime("%a, %d %b %Y %H:%M:%S"))